# Warning
This file is currently a work in progress. Expect instability, inconsistency, and ugly code.
@W

In [1]:
DATA_DIRECTORY = "../../data"

# Delete me, eventually
data_path = DATA_DIRECTORY + '/dialog-babi-task1/'
training_file = data_path + 'dialog-babi-task1-API-calls-trn-workspace.txt'
dev_file = data_path + 'dialog-babi-task1-API-calls-dev-workspace.txt'

In [2]:
from functional import seq

In [3]:
import torch
import torch.nn.functional as F

In [4]:
# Initial object constructors
class Workspace():
    def __init__(self, tree):
        # assert tree is array
        self.tree = tree
        
    def add(address, content):
        assert isinstance(address, (str))
        assert isinstance(content, (str))
        
        def _add(tree):
            if not isinstance(tree, (list)):
                return tree
            elif tree[0] == address:
                return tree.concat([[content]])
            else:
                return map(tree, _add)
        
        return Workspace(_add(self.tree))

    def update(self, action):
        if action.what == 'INIT':
            return Workspace(['root'])
        elif action.what == 'MSG':
            return self
        elif action.what == 'ADD':
            tmp = action.content.split(' ')
            assert (len(tmp) == 2)
            address = tmp[0]
            address = tmp[1]
            return this.add(address, content)
        
    # TODO: Figure out how to flattenDeep    
    def to_words():
        return "NOT IMPLEMENTED YET"
    def to_tree():
        return this.tree
        

In [5]:
class Action():
    def __init__(self, who, what, content=False):
        self.who = who
        self.what = what
        self.content = content
        
    def to_words(self):
        return [f"{self.who}:", self.what] + (self.content.split(' ') if self.content else [])
        
    def to_string(self):
        if self.content:
            return f'{self.who}: {self.what} {self.content}'
        else:
            return f'{self.who}: {self.what}'

In [6]:
def files_to_words(filenames):
     return (seq(filenames)
                .map(lambda filename: open(filename).read().split())
                .flatten()
                .distinct()
                .sorted()
                .to_list())

In [7]:
def string_to_action(s):
    who = s[0]
    i = s.index(" ")
    line = s[i+1:]
    j = line.index(" ")
    what = line[0:j]
    content = line[j+1:]
    return Action(who, what, content)
    

In [8]:
#y = file_to_dialogs(training_file)

In [9]:
def file_to_dialogs(filename):
    init_entry = {
        "workspace": Workspace([]),
        "action": Action(who='A', what='INIT')
    }
    dialog_list = []
    current_workspace = init_entry["workspace"] 
    
    for dialog in open(filename).read().split('\n\n'):
        action_list = [init_entry]
        for action in dialog.split("\n"):
            action_list.append({"workspace": current_workspace,
                                "action": string_to_action(action)})
        dialog_list.append(action_list)
        
    return dialog_list
        
    

In [10]:
special_words = [
  '^', '$',            # start/end markers
  'INIT', 'WAIT',      # action types not in data
  'NO_OUTPUT',         # filler when copy or match word is not available
  'NO_USER_ACTION',    # initial value of lastUserAction
  'NO_WORKSPACE_WORD'
]

In [11]:
def load_babi_data():
    data_path = DATA_DIRECTORY + '/dialog-babi-task1/'
    training_file = data_path + 'dialog-babi-task1-API-calls-trn-workspace.txt'
    dev_file = data_path + 'dialog-babi-task1-API-calls-dev-workspace.txt'
    types_and_tokens = [
            { "type": 'cuisine', "tokens": ['french', 'italian', 'british', 'spanish', 'indian'] },
            { "type": 'location', "tokens": ['rome', 'london', 'bombay', 'paris', 'madrid'] },
            { "type": 'price', "tokens": ['cheap', 'moderate', 'expensive'] },
            { "type": 'people', "tokens": ['two', 'four', 'six', 'eight'] },
        ]
    
    data = {
      "training": file_to_dialogs(training_file),
      "dev": file_to_dialogs(dev_file),
      "words": files_to_words([training_file]) + special_words,
      "types_and_tokens": types_and_tokens
    }

    return data

In [12]:
def create_one_hot(vocab_size):
    return ([[int(i == j) for i in range(vocab_size)] for j in range(vocab_size)])


In [13]:
def action_to_one_hot(action):
    return ((seq(['^'] + action.to_words() + ['$'])
                .map(lambda w: data['words'].index(w))
                .map(lambda i: one_hot[i])
                .to_list()))

In [14]:
def action_to_idx(action):
    return ((seq(['^'] + action.to_words() + ['$'])
                .map(lambda w: data['words'].index(w))
                .to_list()))

In [15]:
MAX_TENSOR_SIZE = 23 #TODO: Refactor to determine automatically
def pad_tensor(tensor):
    m = MAX_TENSOR_SIZE
    a = tensor
    return a + [([0 for i in range(vocab_size)]) for j in range(m - len(a))]

In [16]:
MAX_TENSOR_SIZE = 23 #TODO: Refactor to determine automatically
def pad_word_tensor(tensor):
    m = MAX_TENSOR_SIZE
    a = tensor
    return a + [vocab_size-1 for j in range(m - len(a))]

In [17]:
def actions_to_one_hot(data):
    new_data = (seq(data)
                .map(lambda a: a['action'])
                .map(lambda a: action_to_one_hot(a))
                .map(pad_tensor)
                .map(torch.LongTensor))
    return new_data

                #.map(action_to_one_hot))

In [18]:
def actions_to_idx(data):
    new_data = (seq(data)
                .map(lambda a: a['action'])
                .map(lambda a: action_to_idx(a))
                .map(pad_word_tensor)
                .map(torch.LongTensor))
    return new_data

                #.map(action_to_one_hot))

In [19]:
def dialogs_to_one_hot(dialogs):
    out_data = []
    for dialog in dialogs:
        out = torch.stack(list(actions_to_one_hot(dialog)))
        out_data.append(out)
    return out_data

In [20]:
def dialogs_to_idx(dialogs):
    out_data = []
    for dialog in dialogs:
        out = torch.stack(list(actions_to_idx(dialog)))
        out_data.append(out)
    return out_data

In [21]:
data = load_babi_data()

# Ensure it looks ok
#data['training'][100]["action"].to_string()

In [22]:
data['training'][1][-1]['action'].to_string()

'A: MSG api_call italian london eight cheap'

In [23]:
vocab_size = len(data["words"])
#one_hot = create_one_hot(vocab_size)

In [24]:
# Convert our data into a list of s
#training_data = dialogs_to_one_hot(data['training'])
training_data = (dialogs_to_idx(data['training']))



In [25]:
idx_to_words(training_data[][7])

SyntaxError: invalid syntax (<ipython-input-25-005e7af93950>, line 1)

In [26]:
def idx_to_words(idxs): return list(map(lambda i: data['words'][int(i)], idxs))

# WARNING!
Everything below this cell is currently non-functional. If it compiles, I'll be rather surprised.

In [27]:
import torch
import torch.nn as nn
from torch.autograd import Variable

In [28]:
# Adapted from:
# http://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html

class Encoder(nn.Module):
    def __init__(self, x_size, h_size, n_layers=1):
        super(Encoder, self).__init__()
        self.n_layers = n_layers
        self.h_size = h_size
        self.x_size = x_size
        self.cell = nn.GRU(h_size, h_size)
        self.embedding = nn.Embedding(x_size, h_size)
        
        #self.embedding = nn.Embedding(x_size, h_size)

    # x: input
    # h: hidden state
    def forward(self, input, h):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        for i in range(self.n_layers):
            output, h = self.cell(output, h)
        return output, h
    
    def init_hidden(self):
        return Variable(torch.zeros(1, 1, self.h_size))

In [29]:
class DecoderMock(nn.Module):
    def __init__(self, x_size, h_size, n_layers=1):
        super(DecoderMock, self).__init__()
        self.n_layers = n_layers
        self.h_size = h_size
        self.x_size = x_size
        self.cell = nn.GRU(h_size, h_size)
        self.out = nn.Linear(h_size, x_size)
        self.softmax = nn.LogSoftmax()
        self.embedding = nn.Embedding(x_size, h_size)

    # x: input
    # h: hidden state
    def forward(self, input, h):
        output = self.embedding(input).view(1, 1, -1)
        for i in range(self.n_layers):
            output = F.relu(output)
            output, h = self.cell(output, h)
        output = self.softmax(self.out(output[0]))
        return output, h
    
    def init_hidden(self):
        return Variable(torch.zeros(1, 1, self.h_size))

In [30]:
 class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, n_layers=1):
        super(DecoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax()
        
    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        for i in range(self.n_layers):
            output = F.relu(output)
            output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return Variable(torch.zeros(1, 1, self.hidden_size))

One day, we will be a neural network

In [31]:
training_variables = list(map(lambda d: Variable(d), training_data))

In [32]:
torch.backends.cudnn.enabled = False

# Init Training here

In [33]:
hidden_size = 256
learning_rate = 0.01

In [34]:
e = 1000
encodern = Encoder(vocab_size, hidden_size).cuda()
decodern = DecoderMock(vocab_size, hidden_size).cuda()
plot_losses = []
plot_every = 1
plot_loss_total = 0

encoder_optimizer = torch.optim.SGD(encodern.parameters(), lr=learning_rate)
decoder_optimizer = torch.optim.SGD(decodern.parameters(), lr=learning_rate)
criterion = nn.NLLLoss() 
loss = 0

losses = []


In [35]:
def train_epochs():
    for epoch in range(e):
        plot_loss_total = 0 
        loss = train(encodern, decodern, encoder_optimizer, decoder_optimizer, criterion)
        plot_loss_total += loss[0]

        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0 
        print(plot_loss_avg)
    #showPlot(plot_losses)

In [36]:
import gc

In [ ]:
train_epochs()

DIALOG: 0
DIALOG: 1
DIALOG: 2
DIALOG: 3
DIALOG: 4
DIALOG: 5
DIALOG: 6
DIALOG: 7
DIALOG: 8
DIALOG: 9
DIALOG: 10
DIALOG: 11
DIALOG: 12
DIALOG: 13
DIALOG: 14
DIALOG: 15
DIALOG: 16
DIALOG: 17
DIALOG: 18
DIALOG: 19


In [37]:
def train(encodern, decodern, encoder_optimizer, decoder_optimizer, criterion, offset=0):
   
    encodern.zero_grad()
    decodern.zero_grad()

    encoder_hidden = encodern.init_hidden().cuda()
    decoder_hidden = decodern.init_hidden().cuda()

    TARGET_LENGTH = 20
    BATCH_SIZE = 1

    encoder_outputs = Variable(torch.zeros(1, encodern.h_size) )

    sos = data['words'].index('^')
    eos = data['words'].index('$')

    encoder_input = Variable(torch.LongTensor([sos]).cuda())
    decoder_input = Variable(torch.LongTensor([sos]).cuda())


    loss = 0

    #training_variables = list(map(lambda d: Variable(d), training_data))

    # This may require a little explanation
    # we use history as a cache of messages
    # at every "action" in the dialogs we attempt
    # to predict it given the previously given history
    #for dialog in training_data[offset:BATCH_SIZE+offset]: 
    for it, dialog in enumerate(training_data): 
        gc.collect()
        print("DIALOG: " + str(it))

        # I'm not sure of a better way to optimize this, due to our sequences being variable length
        history = []
        for action in dialog:

            # Append our action to our history
            history = history + [action]

            # If we have only one action we can't have both training and test
            if len(history) < 2: continue

            # Iterate thruugh every message(action) in the history other than the current
            for message in history[:-1]:
                for word in message:
                    encoder_input = Variable(torch.LongTensor([word]).cuda())
                    encoder_outputs, encoder_hidden = encodern(encoder_input, encoder_hidden)

            decoder_hidden = encoder_hidden # Share our hidden state

            # Seed our decoder with the start of sentence token

            # Start Decoding Cycle
            # Note that we use its own predictions as the next input
                # Q: Should I add teacher forcing?
            
            int_d_outputs = []
            for i in range(TARGET_LENGTH):
                decoder_output, decoder_hidden = decodern(decoder_input, decoder_hidden)
                int_d_outputs.append(decoder_output.data)
            
                topv, topi = decoder_output.data.topk(1)
                ni = topi[0][0]
                decoder_input = Variable(torch.LongTensor([[ni]])).cuda()
 
                loss += criterion(decoder_output[-1],
                                  Variable(torch.LongTensor([history[-1][i]]).cuda()))
            
                if ni == eos:
                    break    
                    
            
    loss.backward()
     
    encoder_optimizer.step()
    decoder_optimizer.step() 
    return loss


In [347]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
%matplotlib inline

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    loc = ticker.MultipleLocator(base=0.2) # this locator puts ticks at regular intervals
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)
    

In [208]:
criterion(Variable(a, requires_grad=True),b)

RuntimeError: Variable data has to be a tensor, but got Variable

what is my loss function?

In [ ]:
# TODO: Implement initialization
w

# Whether this is a good idea is still in question
attentional_net = torch.nn.Sequential(
                    torch.nn.Linear(2, 1),
                    torch.nn.Tanh())

# Whether this is a good idea is still in question
decoder_output_net = torch.nn.Sequential(
                        torch.nn.Softmax())

class Decoder(nn.Module):
    def __init__(self, x_size, h_size, n_layers=1):
        self.n_layers = n_layers
        self.h_size = h_size
        self.cell = nn.GRU(x_size, h_size)

        self.out = nn.Linear(h_size, )
        self.attention = torch.nn.Sequential(torch.nn.Linear(2 * x_size, 1),
                                             torch.nn.Tanh())

    def forward(self, decoder_state, encoder_states):
        # Combine State
        combinedState = lambda encoder_state, z: self.attention(torch.cat(decoder_state, encoder_state), 0)

        # This is terrible and I feel dirty
        zero_states = torch.zeros(encoder_states.size()[0])

        # Create attention vector
        attention_vector = T.softmax(torch.Tensor.map_(encoder_states, zero_states,
                                                       combinedState))

        final = encoder_states * torch.transpose(attention_vector.unsqueeze(0), 0, 1).expand(encoder_states.size())

        for i in range(self.n_layers):
            output, h = self.cell(output, h)
        return output, h

In [427]:
def go(e):
    encodern = Encoder(vocab_size, hidden_size).cuda()
    decodern = DecoderMock(vocab_size, hidden_size).cuda()
    plot_losses = []
    plot_every = 1
    plot_loss_total = 0
    
    encoder_optimizer = torch.optim.SGD(encodern.parameters(), lr=learning_rate)
    decoder_optimizer = torch.optim.SGD(decodern.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss() 
    loss = 0
    
    losses = []
    for epoch in range(e):
        loss = build_and_train(encodern, decodern, encoder_optimizer, decoder_optimizer, criterion)
        plot_loss_total += loss[0]
        
        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0 
        print(plot_loss_avg)
    #showPlot(plot_losses)
    
 

In [428]:
build_and_train()

TypeError: build_and_train() missing 5 required positional arguments: 'encodern', 'decodern', 'encoder_optimizer', 'decoder_optimizer', and 'criterion'